In [ ]:
import TraceDisplay
from TraceDisplay import BOKEH_RENDERER
from IPython.display import HTML, display
import pandas as pd
pd.set_option('max_colwidth', 999)

In [ ]:
image = ! find ${WALK_PATH:-.} -name '*.i.h5'
data_path = image[0]
html_path = '/dev/null'

In [ ]:
image = TraceDisplay.Image()
image.load(data_path)

In [ ]:
bkr = TraceDisplay.BokehRenderer()
bkr.show()

In [ ]:
bkr.render(image)

In [ ]:
# Create a snapshot before calling filter
# bkr.image
display(HTML(bkr.to_html()))
display(image.filter)
display(image['/sched_switch'])

In [ ]:
pid = image['/sched_process_exec'].iloc[0]['common_pid']
display(pid)
image.filter.update({
    k:f"common_pid == {pid}"
    for k in image
})

In [ ]:
# Simulate Button click
bkr.update()
# Create a snapshot after calling filter
# bkr.image
display(HTML(bkr.to_html()))
display(image.filter)
display(image['/sched_switch'])

In [ ]:
# Append custom categories

image.category.append(
    label=f'pid == {pid}',
    color='#FF0000',
    query = {
        k:f"common_pid == {pid}"
        for k in image
    },
)

image.category.append(
    label=f'pid != {pid}',
    color='#00FF00',
    query = {
        k:f"common_pid != {pid}"
        for k in image
    },
)

image.filter.clear()
bkr.update()
display(HTML(bkr.to_html()))
image.category

In [ ]:
# Create a custom shape
# Example: migrations
#
import numpy as np
display(image['/sched_migrate_task'])

image['/migrations'] = pd.DataFrame({
    'x0' : np.array(image['/sched_migrate_task'].index, dtype=float),
    'x1' : np.array(image['/sched_migrate_task'].index, dtype=float),
    'y0' : np.array(image['/sched_migrate_task']['orig_cpu'], dtype=float),
    'y1' : np.array(image['/sched_migrate_task']['dest_cpu'], dtype=float),
    'category' : np.zeros(len(image['/sched_migrate_task']), dtype=int),
    'pid' : np.array(image['/sched_migrate_task']['pid'], dtype=int),
})

display(image['/migrations'])

# Before drawing we need to set the shape and create some categories
# Let's create per_pid categories with random colors
image.shape['/migrations'] = 'LineShape' # Only shape available (we could draw points)
import random
r = lambda: random.randint(0,255)
random_color = lambda : '#%02X%02X%02X' % (r(),r(),r())
for pid in np.unique(image['/migrations']['pid']):
    image.category.append(
        label=f'/migrations pid=={pid}',
        color=random_color(),
        query={'/migrations':f'pid=={pid}'},
    )

# Cancel the two previous categories 
image.category[7].query = '{}'
image.category[8].query = '{}'

# Draw
bkr.update()
display(HTML(bkr.to_html()))
display(image.category)